# Imports

In [7]:
import numpy as np
import pandas as pd
import sqlalchemy

import os

from datetime import datetime

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# EDA

In [22]:
df = pd.read_csv('Datasets/Cleaned/clean_benign.csv', index_col=0, nrows=100000)
df.shape

(100000, 83)

In [23]:
df.columns

Index(['Flow_ID', 'Flow_Duration', 'Fwd_Total_Pkts', 'Bwd_Total_Pkts',
       'Fwd_Total_Bytes', 'Bwd_Total_Bytes', 'Fwd_Pkt_Length_Max',
       'Fwd_Pkt_Length_Min', 'Fwd_Pkt_Length_Mean', 'Fwd_Pkt_Length_Std',
       'Bwd_Pkt_Length_Max', 'Bwd_Pkt_Length_Min', 'Bwd_Pkt_Length_Mean',
       'Bwd_Pkt_Length_Std', 'Flow_Bytes_Sec', 'Flow_Pkts_Sec',
       'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min',
       'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max',
       'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std',
       'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags',
       'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length',
       'Bwd_Header_Length', 'Fwd_Pkts_Sec', 'Bwd_Pkts_Sec', 'Pkt_Length_Min',
       'Pkt_Length_Max', 'Pkt_Length_Mean', 'Pkt_Length_Std', 'Pkt_Length_Var',
       'FIN_Flag_Count', 'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count',
       'ACK_Flag_Count', 'URG_Flag_Count', 'CWE_Flag_Count', 'ECE_Fl

In [24]:
df.head()

,Flow_ID,Flow_Duration,Fwd_Total_Pkts,Bwd_Total_Pkts,Fwd_Total_Bytes,Bwd_Total_Bytes,Fwd_Pkt_Length_Max,Fwd_Pkt_Length_Min,Fwd_Pkt_Length_Mean,Fwd_Pkt_Length_Std,...,Time_Active_Min,Time_Idle_Mean,Time_Idle_Std,Time_Idle_Max,Time_Idle_Min,Label,HOPOPT,TCP,UDP,Malicious
0,172.19.1.46-10.200.7.7-52422-3128-6,45523,22,55,132,110414.0,6,6,6.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Benign_HTTP_PROXY,0,1,0,0
1,172.19.1.46-10.200.7.7-52422-3128-6,1,2,0,12,0.0,6,6,6.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Benign_HTTP_PROXY,0,1,0,0
2,10.200.7.217-50.31.185.39-38848-80-6,1,3,0,674,0.0,337,0,224.666667,194.567041,...,0.0,0.0,0.0,0.0,0.0,Benign_HTTP,0,1,0,0
3,10.200.7.217-50.31.185.39-38848-80-6,217,1,3,0,0.0,0,0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,Benign_HTTP,0,1,0,0
4,192.168.72.43-10.200.7.7-55961-3128-6,78068,5,0,1076,0.0,529,6,215.200000,286.458898,...,0.0,0.0,0.0,0.0,0.0,Benign_HTTP_PROXY,0,1,0,0


In [25]:
engine = sqlalchemy.create_engine('sqlite:///test.sqlite')

In [26]:
df.to_sql(name='test', con=engine, if_exists='replace')